In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from tqdm import tqdm


### tipo_carro: 
* Antigo: 0
* Buggy: 1
* Caminhão Leve: 2
* Conversível: 3
* Hatch: 4
* Passeio: 5
* Pick-up: 6
* Sedã: 7
* SUV: 8
* Van/Utilitário: 9
### combustivel:
* Gasolina: 0
* Álcool: 1
* Flex: 2
* Diesel: 3
* Híbrido: 4
* Elétrico: 5
### gnv:
* Sim: 1
* Não: 0
### cambio:
* Manual: 0
* Automático: 1
* Semi-automático: 2

### Direçao

* Hidráulica: 0
* Mecânica: 1
* Elétrica: 2
* Assistida: 3

In [ ]:
service = Service()
options = webdriver.ChromeOptions()

#headless
# options.add_argument('--headless')

chrome_prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", chrome_prefs)

options.page_load_strategy = 'eager'
driver = webdriver.Chrome(service=service, options=options)

df = pd.DataFrame(columns=['link','nome_anuncio', 'desc', 'orig_preco', 'preco', 'categoria', 'modelo', 'marca', 'tipo_carro', 'ano', 'quilometragem', 'cilindrada'
                           , 'combustivel', 'gnv', 'cambio', 'cor', 'portas', 'fim_placa', 'direcao', 'airbag', 'alarme', 'ar_condicionado'
                           , 'trava_eletrica', 'vidro_eletrico', 'som', 'sensor_re', 'camera_re', 'blindado', 'estado'])

def pega_elemento(elem_xpath):
    try:
        elem = driver.find_element(By.XPATH, elem_xpath)
        return elem.text
    except:
        return ''

# driver.get('https://pe.olx.com.br/grande-recife/autos-e-pecas/carros-vans-e-utilitarios/golf-gl-1-8-gasolina-1237337170')

def raspa_links(j, i, estado, passo):
    if j >= 150000:
        if i == 1:
            url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-{}?ps={}'.format(estado,j)
        else:
            url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-{}?ps={}&o={}'.format(estado,j,i)
    else:
        if i == 1:
            url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-{}?pe={}&ps={}'.format(estado,j, j-passo)
        else:
            url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/estado-{}?pe={}&ps={}&o={}'.format(estado,j, j-passo,i)

    try:
        driver.get(url)
        try:
            botao_novidades = '/html/body/div[4]/div/div/div[1]/div/div/div/div/button'
            botao_novidades = driver.find_element(By.XPATH, botao_novidades)
            botao_novidades.click()
        except:
            pass

        class_links = driver.find_elements(By.CLASS_NAME, 'olx-ad-card__link-wrapper')
        links = [link.get_attribute('href') for link in class_links]
    except:
        pass

    return links

def scrape(url, olx_estado):
    try:
        driver.get(url)
        elem_nome_anuncio = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[19]/div/div/h1'
        elem_container_info = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[34]/div/div/div/div[4]'
        elem_container_loc = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[39]/div/div/div/div[1]'
        elem_opcionais = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[34]/div/div/div/div[5]/div[2]'

        elem_orig_preco = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[20]/div/div/div/div/div[1]/span'
        elem_preco = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[20]/div/div/div/div/div[1]/h2'

        elem_desc = '/html/body/div[2]/div/div[4]/div[2]/div/div[2]/div[1]/div[28]/div/div/div/div[2]/div/p/span'

        keys = []
        values = []

        keys.append('link')
        values.append(url)

        nome = pega_elemento(elem_nome_anuncio)
        keys.append('nome_anuncio')
        values.append(nome)

        desc = pega_elemento(elem_desc)
        #cleaning desc
        try:
            desc = desc.replace('\n', ' ')
            desc = desc.replace('\t', ' ')
            desc = desc.replace('\r', ' ')
            desc = desc.replace(';', '')
        except:
            pass
        if desc == '':
            desc = None
        keys.append('desc')
        values.append(desc)

        orig_preco = pega_elemento(elem_orig_preco)
        keys.append('orig_preco')
        values.append(orig_preco)

        
        preco = pega_elemento(elem_preco)
        keys.append('preco')
        values.append(preco)

        container = pega_elemento(elem_container_info)

        for i in container.split('\n'):
            #se o índice é par, é uma chave
            if container.split('\n').index(i) % 2 == 0:
                keys.append(i)
            #senão, é um valor
            else:
                values.append(i)

        #renomear keys para o padrão do dataframe

        
        for i in range(len(keys)):
            if keys[i] == 'Categoria':
                keys[i] = 'categoria'
            if keys[i] == 'Modelo':
                keys[i] = 'modelo'
            if keys[i] == 'Marca':
                keys[i] = 'marca'
            if keys[i] == 'Tipo de veículo':
                keys[i] = 'tipo_carro'
            if keys[i] == 'Ano':
                keys[i] = 'ano'
            if keys[i] == 'Quilometragem':
                keys[i] = 'quilometragem'
            if keys[i] == 'Potência do motor':
                keys[i] = 'cilindrada'
            if keys[i] == 'Combustível':
                keys[i] = 'combustivel'
            if keys[i] == 'Possui Kit GNV':
                keys[i] = 'gnv'
            if keys[i] == 'Câmbio':
                keys[i] = 'cambio'
            if keys[i] == 'Cor':
                keys[i] = 'cor'
            if keys[i] == 'Portas':
                keys[i] = 'portas'
            if keys[i] == 'Final de placa': 
                keys[i] = 'fim_placa'
            if keys[i] == 'Tipo de direção':
                keys[i] = 'direcao'

        #renomear valores de dados categóricos para numéricos

        for i in values:
            #tipo_carro
            if i == 'Antigo':
                values[values.index(i)] = 0
            if i == 'Buggy':
                values[values.index(i)] = 1
            if i == 'Caminhão Leve':
                values[values.index(i)] = 2
            if i == 'Conversível':
                values[values.index(i)] = 3
            if i == 'Hatch':
                values[values.index(i)] = 4
            if i == 'Passeio':
                values[values.index(i)] = 5
            if i == 'Pick-up':
                values[values.index(i)] = 6
            if i == 'Sedã':
                values[values.index(i)] = 7
            if i == 'SUV':
                values[values.index(i)] = 8
            if i == 'Van/Utilitário':
                values[values.index(i)] = 9

            #combustivel
            if i == 'Gasolina':
                values[values.index(i)] = 0
            if i == 'Álcool':
                values[values.index(i)] = 1
            if i == 'Flex':
                values[values.index(i)] = 2
            if i == 'Diesel':
                values[values.index(i)] = 3
            if i == 'Híbrido':
                values[values.index(i)] = 4
            if i == 'Elétrico':
                values[values.index(i)] = 5
            if i == 'Gás Natural':
                values[values.index(i)] = 6
            #gnv
            if i == 'Sim':
                values[values.index(i)] = 1
            if i == 'Não':
                values[values.index(i)] = 0
            #cambio
            if i == 'Manual':
                values[values.index(i)] = 0
            if i == 'Automático':
                values[values.index(i)] = 1
            if i == 'Semi-automático':
                values[values.index(i)] = 2
            if i == 'Semi-Automático':
                values[values.index(i)] = 2 #mesma coisa, muda só o case
            #cor
            if i == 'Preto':
                values[values.index(i)] = 0
            if i == 'Branco':
                values[values.index(i)] = 1
            if i == 'Prata':
                values[values.index(i)] = 2
            if i == 'Vermelho':
                values[values.index(i)] = 3
            if i == 'Cinza':
                values[values.index(i)] = 4
            if i == 'Azul':
                values[values.index(i)] = 5
            if i == 'Amarelo':
                values[values.index(i)] = 6
            if i == 'Verde':
                values[values.index(i)] = 7
            if i == 'Laranja':
                values[values.index(i)] = 8
            if i == 'Outra':
                values[values.index(i)] = 9
            #portas
            if i == '2 portas':
                values[values.index(i)] = 0
            if i == '4 portas':
                values[values.index(i)] = 1
            #tipo de direção
            if i == 'Hidráulica':
                values[values.index(i)] = 0
            if i == 'Mecânica':
                values[values.index(i)] = 1
            if i == 'Elétrica':
                values[values.index(i)] = 2
            if i == 'Assistida':
                values[values.index(i)] = 3
        
            
        airbag = 0
        alarme = 0
        ar_condicionado = 0
        trava_eletrica = 0
        vidro_eletrico = 0
        som = 0
        sensor_re = 0
        camera_re = 0
        blindado = 0

        try:
            elem_opcionais = driver.find_element(By.XPATH, elem_opcionais)
            for i in elem_opcionais.text.split('\n'):
                if i == 'Air bag':
                    airbag = 1
                if i == 'Alarme':
                    alarme = 1
                if i == 'Ar condicionado':
                    ar_condicionado = 1
                if i == 'Trava elétrica':
                    trava_eletrica = 1
                if i == 'Vidro elétrico':
                    vidro_eletrico = 1
                if i == 'Som':
                    som = 1
                if i == 'Sensor de ré':
                    sensor_re = 1
                if i == 'Câmera de ré':
                    camera_re = 1
                if i == 'Blindado':
                    blindado = 1
        except:
            pass

        keys.append('airbag')
        values.append(airbag)
        keys.append('alarme')
        values.append(alarme)
        keys.append('ar_condicionado')
        values.append(ar_condicionado)
        keys.append('trava_eletrica')
        values.append(trava_eletrica)
        keys.append('vidro_eletrico')
        values.append(vidro_eletrico)
        keys.append('som')
        values.append(som)
        keys.append('sensor_re')
        values.append(sensor_re)
        keys.append('camera_re')
        values.append(camera_re)
        keys.append('blindado')
        values.append(blindado)

        keys.append('estado')
        values.append(olx_estado)

        new_row = {}
        #construir o dicionário
        for i, j in zip(keys, values):
            new_row[i] = j

        return new_row
    except:
        pass
# for i in tqdm(pd.read_excel('links_olx_pe.xlsx')['link']):

ufs = ['ac', 'al', 'ap', 'am', 'ba', 'ce', 'df', 'es', 'go', 'ma', 'mt', 'ms', 'mg', 'pa', 'pb', 'pr', 'pe', 'pi',
       'rj', 'rn', 'rs', 'ro', 'rr', 'sc', 'sp', 'se', 'to']

import time

#convert seconds to hours, minutes, seconds
print(time.strftime("%H:%M:%S", time.gmtime(time.time())))

#print chromedriver version
print(webdriver.__version__)

passo = 1000

for j in range(1000, 150001, passo):
    for i in range(1, 101):
        links_list = raspa_links(j, i, 'rj', passo)
        if len(links_list) == 0:
            print('Página {} do range {} não tem links'.format(i, j))
            break
            
        for links in tqdm(links_list, desc='Página {} do range {}'.format(i, j)):
            df.loc[len(df)] = scrape(links, 'rj')

df.dropna(subset=['modelo'])

driver.quit()

print(time.strftime("%H:%M:%S", time.gmtime(time.time())))


In [109]:
# service = Service()
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome(service=service, options=options)

# def get_element(elem_xpath):
#     try:    
#         elem = driver.find_elements(By.XPATH, elem_xpath)
#         return elem.text
#     #print error if element not found
#     except:
#         return ''

# driver.get(dfdfdf.link[19])

# elem_nome_anuncio = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[20]/div/div/div/div[1]/div[1]/h2' 
                    

# elem_container_info = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[34]/div/div/div/div[4]'

# elem_container_loc = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[39]/div/div/div/div[1]'
# elem_opcionais = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[34]/div/div/div/div[5]/div[2]/div[2]'

# elem_orig_preco = '//*[@id="content"]/div[2]/div/div[2]/div[1]/div[20]/div/div/div/div[1]/div[1]/h2'
# elem_preco = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[20]/div/div/div/div[1]/div[1]/h2'

# elem_desc = '/html/body/div[2]/div/div[3]/div[2]/div/div[2]/div[1]/div[28]/div/div/div/div[2]/div/p/span'

# keys = []
# values = []

# keys = []
# values = []

# keys.append('link')
# values.append(dfdfdf.link[19])

# nome = get_element(elem_nome_anuncio)
# keys.append('nome_anuncio')
# values.append(nome)

# desc = get_element(elem_desc)
# #cleaning desc

# desc = desc.replace('\n', ' ')
# desc = desc.replace('\t', ' ')
# desc = desc.replace('\r', ' ')
# desc = desc.replace(';', '')

# if desc == '':
#     desc = None
# keys.append('desc')
# values.append(desc)

# orig_preco = get_element(elem_orig_preco)
# keys.append('orig_preco')
# values.append(orig_preco)


# preco = get_element(elem_preco)
# keys.append('preco')
# values.append(preco)

# container = get_element(elem_container_info)
# loc_container = get_element(elem_container_loc)

# for i in container.split('\n'):
#     #se o índice é par, é uma chave
#     if container.split('\n').index(i) % 2 == 0:
#         keys.append(i)
#     #senão, é um valor
#     else:
#         values.append(i)

# #renomear keys para o padrão do dataframe


# for i in range(len(keys)):
#     if keys[i] == 'Categoria':
#         keys[i] = 'categoria'
#     if keys[i] == 'Modelo':
#         keys[i] = 'modelo'
#     if keys[i] == 'Marca':
#         keys[i] = 'marca'
#     if keys[i] == 'Tipo de veículo':
#         keys[i] = 'tipo_carro'
#     if keys[i] == 'Ano':
#         keys[i] = 'ano'
#     if keys[i] == 'Quilometragem':
#         keys[i] = 'mileage'
#     if keys[i] == 'Potência do motor':
#         keys[i] = 'potencia'
#     if keys[i] == 'Combustível':
#         keys[i] = 'combustivel'
#     if keys[i] == 'Possui Kit GNV':
#         keys[i] = 'gnv'
#     if keys[i] == 'Câmbio':
#         keys[i] = 'cambio'
#     if keys[i] == 'Cor':
#         keys[i] = 'cor'
#     if keys[i] == 'Portas':
#         keys[i] = 'portas'
#     if keys[i] == 'Final de placa': 
#         keys[i] = 'fim_placa'
#     if keys[i] == 'Tipo de direção':
#         keys[i] = 'direcao'

# #renomear valores de dados categóricos para numéricos

# for i in values:
#     #tipo_carro
#     if i == 'Antigo':
#         values[values.index(i)] = 0
#     if i == 'Buggy':
#         values[values.index(i)] = 1
#     if i == 'Caminhão Leve':
#         values[values.index(i)] = 2
#     if i == 'Conversível':
#         values[values.index(i)] = 3
#     if i == 'Hatch':
#         values[values.index(i)] = 4
#     if i == 'Passeio':
#         values[values.index(i)] = 5
#     if i == 'Pick-up':
#         values[values.index(i)] = 6
#     if i == 'Sedã':
#         values[values.index(i)] = 7
#     if i == 'SUV':
#         values[values.index(i)] = 8
#     if i == 'Van/Utilitário':
#         values[values.index(i)] = 9

#     #combustivel
#     if i == 'Gasolina':
#         values[values.index(i)] = 0
#     if i == 'Álcool':
#         values[values.index(i)] = 1
#     if i == 'Flex':
#         values[values.index(i)] = 2
#     if i == 'Diesel':
#         values[values.index(i)] = 3
#     if i == 'Híbrido':
#         values[values.index(i)] = 4
#     if i == 'Elétrico':
#         values[values.index(i)] = 5
#     #gnv
#     if i == 'Sim':
#         values[values.index(i)] = 1
#     if i == 'Não':
#         values[values.index(i)] = 0
#     #cambio
#     if i == 'Manual':
#         values[values.index(i)] = 0
#     if i == 'Automático':
#         values[values.index(i)] = 1
#     if i == 'Semi-automático':
#         values[values.index(i)] = 2
#     #cor
#     if i == 'Preto':
#         values[values.index(i)] = 0
#     if i == 'Branco':
#         values[values.index(i)] = 1
#     if i == 'Prata':
#         values[values.index(i)] = 2
#     if i == 'Vermelho':
#         values[values.index(i)] = 3
#     if i == 'Cinza':
#         values[values.index(i)] = 4
#     if i == 'Azul':
#         values[values.index(i)] = 5
#     if i == 'Amarelo':
#         values[values.index(i)] = 6
#     if i == 'Verde':
#         values[values.index(i)] = 7
#     if i == 'Laranja':
#         values[values.index(i)] = 8
#     if i == 'Outra':
#         values[values.index(i)] = 9
#     #portas
#     if i == '2 portas':
#         values[values.index(i)] = 0
#     if i == '4 portas':
#         values[values.index(i)] = 1
#     #tipo de direção
#     if i == 'Hidráulica':
#         values[values.index(i)] = 0
#     if i == 'Mecânica':
#         values[values.index(i)] = 1
#     if i == 'Elétrica':
#         values[values.index(i)] = 2
    
    

# airbag = 0
# alarme = 0
# ar_condicionado = 0
# trava_eletrica = 0
# vidro_eletrico = 0
# som = 0
# sensor_re = 0
# camera_re = 0
# blindado = 0


# opcionais = get_element(elem_opcionais)

# for i in opcionais.split('\n'):
#     if i == 'Air bag':
#         airbag = 1
#     if i == 'Alarme':
#         alarme = 1
#     if i == 'Ar condicionado':
#         ar_condicionado = 1
#     if i == 'Trava elétrica':
#         trava_eletrica = 1
#     if i == 'Vidro elétrico':
#         vidro_eletrico = 1
#     if i == 'Som':
#         som = 1
#     if i == 'Sensor de ré':
#         sensor_re = 1
#     if i == 'Câmera de ré':
#         camera_re = 1
#     if i == 'Blindado':
#         blindado = 1

# keys.append('airbag')
# values.append(airbag)
# keys.append('alarme')
# values.append(alarme)
# keys.append('ar_condicionado')
# values.append(ar_condicionado)
# keys.append('trava_eletrica')
# values.append(trava_eletrica)
# keys.append('vidro_eletrico')
# values.append(vidro_eletrico)
# keys.append('som')
# values.append(som)
# keys.append('sensor_re')
# values.append(sensor_re)
# keys.append('camera_re')
# values.append(camera_re)
# keys.append('blindado')
# values.append(blindado)

# keys.append('estado')
# values.append('PE')

# for i in loc_container.split('\n'):
#     if i == 'Município':
#         keys.append('municipio')
#         values.append(loc_container.split('\n')[loc_container.split('\n').index(i)+1])

# new_row = {}
# #construir o dicionário
# for i, j in zip(keys, values):
#     new_row[i] = j

# # driver.quit()

# print(keys)
# print(values)
# print(new_row)


['link', 'nome_anuncio', 'desc', 'orig_preco', 'preco', '', 'airbag', 'alarme', 'ar_condicionado', 'trava_eletrica', 'vidro_eletrico', 'som', 'sensor_re', 'camera_re', 'blindado', 'estado']
['https://pe.olx.com.br/grande-recife/autos-e-pecas/carros-vans-e-utilitarios/strada-endurance-1-4-flex-2023-0km-locacao-flua-fiat-1215714305', '', None, '', '', 0, 0, 0, 0, 0, 0, 0, 0, 0, 'PE']
{'link': 'https://pe.olx.com.br/grande-recife/autos-e-pecas/carros-vans-e-utilitarios/strada-endurance-1-4-flex-2023-0km-locacao-flua-fiat-1215714305', 'nome_anuncio': '', 'desc': None, 'orig_preco': '', 'preco': '', '': 0, 'airbag': 0, 'alarme': 0, 'ar_condicionado': 0, 'trava_eletrica': 0, 'vidro_eletrico': 0, 'som': 0, 'sensor_re': 0, 'camera_re': 0, 'blindado': 'PE'}
